In [ ]:
from os.path import basename, exists

def download(url):
    filename = basename(url)
    if not exists(filename):
        from urllib.request import urlretrieve

        local, _ = urlretrieve(url, filename)
        print("Downloaded " + local)

download("https://github.com/AllenDowney/ThinkStats2/raw/master/code/thinkstats2.py")
download("https://github.com/AllenDowney/ThinkStats2/raw/master/code/thinkplot.py")
download("https://github.com/AllenDowney/ThinkStats2/raw/master/code/nsfg.py")

# Load libraries
import thinkstats2
import nsfg
import numpy as np
import random
import pandas as pd

# Load the NSFG data
data = nsfg.ReadFemPreg()

# Ensure birthord column has no NaN values
data = data.dropna(subset=['birthord'])

# Function to calculate the difference in means between first babies and others
def DiffMeans(data):
    firsts = data[data.birthord == 1].prglngth
    others = data[data.birthord != 1].prglngth
    return firsts.mean() - others.mean()

# Function to permute the birthord column and calculate the difference in means
def PermuteDiffMeans(data):
    # Shuffle the birthord column
    shuffled_birthord = np.random.permutation(data.birthord.values)
    # Create a copy of the data and assign the shuffled birthord
    shuffled_data = data.copy()
    shuffled_data.loc[:, 'birthord'] = shuffled_birthord  # Safe modification
    # Calculate the difference in means
    return DiffMeans(shuffled_data)

# Function to run hypothesis tests with different sample sizes
def RunTests(data, sample_sizes, num_tests=100):
    results = {}
    for size in sample_sizes:
        p_values = []
        for _ in range(num_tests):
            # Sample rows safely
            sample = data.sample(n=size, random_state=None)
            # Calculate the observed difference in means
            diff = DiffMeans(sample)
            # Generate permuted differences
            permuted_diffs = [PermuteDiffMeans(sample) for _ in range(1000)]
            # Calculate the p-value
            p_value = np.mean(np.abs(permuted_diffs) >= np.abs(diff))
            p_values.append(p_value)
        results[size] = p_values
    return results

# Define sample sizes to test
sample_sizes = [1000, 500, 200, 100, 50, 20]

# Run the tests
results = RunTests(data, sample_sizes)

# Analyze the results
for size, p_values in results.items():
    print(f"Sample Size: {size}, Mean P-value: {np.mean(p_values)}")
    print(f"Proportion of Significant Tests (p < 0.05): {np.mean(np.array(p_values) < 0.05)}")

Downloaded nsfg.py
